In [1]:
from vedo import dataurl, Mesh, Plotter, Volume, settings, LinearTransform, Box, Line
import numpy as np
import pandas as pd
from normalize import normalize_shape,get_eigenvectors, get_center_of_mass,normalize_vertices,normalize_scale
import os
from pathlib import Path
import matplotlib.pyplot as plt
settings.default_backend = 'k3d'
np.set_printoptions(suppress=True)


In [2]:

def normalize_position(mesh:Mesh,inplace=True):
    """
    Shifts input mesh so that its center of mass coincides with the origin
    ----------------------------
    Args:
        Vedo Mesh
    Returns:
        Vedo Mesh
    """
    if(not inplace):
        wMesh=mesh.copy()
    else:
        wMesh=mesh
    LT = LinearTransform()
    LT.translate(wMesh.transform.position-wMesh.center_of_mass())
    LT.move(wMesh)
    return wMesh

def normalize_pose(mesh:Mesh,inplace=True):
    """
    Rotates mesh so that its major axes are aligned with the unit axes. 
    ----------------------------
    Input:
        Vedo Mesh
    Returns:
        Vedo Mesh
    """
    if(not inplace):
        wMesh=mesh.copy()
    else:
        wMesh=mesh
    eigenvectors,eigenvalues = get_eigenvectors(wMesh)
    ranking = np.argpartition(eigenvalues, 2)
    aligned_matrix = [eigenvectors[ranking[2]],
                      eigenvectors[ranking[1]],
                      np.cross(eigenvectors[ranking[2]],eigenvectors[ranking[1]])]
    #wMesh.vertices = np.dot(wMesh.vertices-mesh.center_of_mass(),np.transpose(aligned_matrix))

    wMesh.vertices = np.dot(aligned_matrix,wMesh.vertices.T).T
    return wMesh
    verts = []
    c = get_center_of_mass(mesh)
    for i in range(0, len(mesh.vertices)):
        v = mesh.vertices[i]
        p1 = np.dot(v-c, eigenvectors[ranking[2]])
        p2 = np.dot(v-c, eigenvectors[ranking[1]])
        p3 = np.cross(eigenvectors[ranking])
        verts.append([p1, p2, p3])
    wMesh.vertices = verts
    return wMesh

def calc_eigenvectors(mesh):
    """
    It computes eigenvectors from a set of vertices.
    ----------------------------
    Args:
       verts (obj: 'list' of obj: 'list' of float): The list of vertices

    Returns:
        eigenvalues (obj: 'list'): The list of eigenvalues
        eigenvectors (obj: 'list'): The list of eigenvectors

    """
    A = np.zeros((3, len(mesh.vertices)))

    A[0] = np.array([x[0] for x in mesh.vertices]) # First row is all the X_coords
    A[1] = np.array([x[1] for x in mesh.vertices]) # second row is all the Y_coords
    A[2] = np.array([x[2] for x in mesh.vertices]) # third row is all the z-coords
    
    A_cov = np.cov(A) # This is returns a 3x3
    eigenvalues, eigenvectors = np.linalg.eigh(A_cov)

    return eigenvectors,eigenvalues


In [3]:
#Rich Display

z_axis = Line([0,0,0], [0,0,2], lw=3).c("Blue")
y_axis = Line([0,0,0], [0,2,0], lw=3).c("Green")
x_axis = Line([0,0,0], [2,0,0], lw=3).c("Red")
unit_box = Box(width=1,height=1,length=1).c("Black").wireframe(True)

filepath = "../shapes/FloorLamp/m619.obj"
mesh = Mesh(filepath)
#normalize_position(mesh)
print(mesh)

# Set the backcolor of the mesh to violet
# and show edges with a linewidth of 2
#mesh_v = mesh.copy(deep=True).c("Red")

mesh_v2 = mesh.copy(deep=True).c("Red")
normalize_vertices(mesh_v2)
normalize_position(mesh_v2)

#mesh_v3 = normalize_position(mesh_v2,inplace=False).c("Red")#.normalize_position()
#mesh_v4 = normalize_pose(mesh_v3,inplace=False).c("Red")


#mesh2 = normalize_position(mesh,inplace=False).c("black")

#print(mesh2)
#mesh3 = normalize_pose(mesh2,inplace=False).c("Green")

eigenvectors, eigenvals = calc_eigenvectors(mesh_v2)

z_axis = Line([0,0,0], [0,0,2], lw=3).c("Blue")
y_axis = Line([0,0,0], [0,2,0], lw=3).c("Green")
x_axis = Line([0,0,0], [2,0,0], lw=3).c("Red")

#print(mesh_v4)
print("EIGENVECTORS:",eigenvectors,eigenvals)
ranking = np.argpartition(eigenvals, 2)
eig1 = Line([0,0,0], eigenvectors[ranking[2]], lw=3).c("Magenta")
eig2 = Line([0,0,0], eigenvectors[ranking[1]], lw=3).c("Magenta")
eig3 = Line([0,0,0], eigenvectors[ranking[0]], lw=3).c("Magenta")

eigenvectors, eigenvals = get_eigenvectors(mesh)
print("EIGENVECTORS2:",eigenvectors,eigenvals)

eig1_1 = Line([0,0,0], eigenvectors[ranking[2]], lw=3).c("Green")
eig2_1 = Line([0,0,0], eigenvectors[ranking[1]], lw=3).c("Green")
eig3_1 = Line([0,0,0], eigenvectors[ranking[0]], lw=3).c("Green")


plot = Plotter(bg='white')
#plot.show(mesh_v3,mesh2,x_axis,y_axis,z_axis,unit_box,eig1,eig2,eig3)
plot.show(mesh_v2,x_axis,y_axis,z_axis,unit_box,eig1,eig2,eig3,eig1_1,eig2_1,eig3_1)

vedo.mesh.Mesh at (0x14e72c5dbc0)                                          
name          : Mesh
file name     : ../shapes/FloorLamp/m619.obj
elements      : vertices=361 polygons=696 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.133367, diagonal=0.973164
center of mass: (0.0996223, 0.828652, 0.0996225)
bounds        : x=(0.0250, 0.174), y=(0.0250, 0.975), z=(0.0250, 0.174)
pointdata     : "Normals" (float32), dim=3
celldata      : "GroupIds" (float32), dim=1, range=(0, 0)
EIGENVECTORS: [[ 0.75242089 -0.65868262  0.00002896]
 [-0.00081075 -0.0009701  -0.9999992 ]
 [ 0.65868212  0.75242027 -0.00126395]] [0.00031382 0.00036382 0.05069214]
EIGENVECTORS2: [[-0.00000001  1.          0.00000132]
 [ 1.          0.00000001  0.00000007]
 [-0.00000007 -0.00000132  1.        ]] [0.0487125  0.00138915 0.00138915]


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [5]:
print(mesh_v2.filename)

../shapes/FloorLamp/m619.obj


In [34]:
print(mesh)
print(mesh_v2)

vedo.mesh.Mesh at (0x196a7122ea0)                                          
name          : Mesh
file name     : ../shapes\FloorLamp\m619.obj
elements      : vertices=361 polygons=696 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.133367, diagonal=0.973164
center of mass: (0.0996223, 0.828652, 0.0996225)
bounds        : x=(0.0250, 0.174), y=(0.0250, 0.975), z=(0.0250, 0.174)
pointdata     : "Normals" (float32), dim=3
celldata      : "GroupIds" (float32), dim=1, range=(0, 0)
vedo.mesh.Mesh at (0x196a7123ca0)                                          
name          : Mesh
file name     : ../shapes\FloorLamp\m619.obj
color         : red, rgb=(0.996, 0.118, 0.122), alpha=1.0
elements      : vertices=18,025 polygons=36,024 lines=0
position      : (0, 0, 0)
scaling       : (1.00000, 1.00000, 1.00000)
size          : average=0.198285, diagonal=0.973164
center of mass: (0.0996227, 0.440077, 0.0996249)
bounds        : x=(0.0250, 0.174), y=

In [58]:
def get_vertex_dist(mesh:Mesh):
    #As the ratio of the smallest cell against largest cell
    minar = np.inf
    maxar = 0
    for idx in range(0,len(mesh.vertices)):
        ar = 0
        for cp in mesh.connected_vertices(idx):
            ar+= np.abs(np.linalg.norm(mesh.vertices[idx] - mesh.vertices[cp]))
        

        if ar!=0 and ar<minar:
            minar=ar
        elif ar!=0 and ar>maxar:
            maxar=ar
        #print(ar,minar,maxar)

    return maxar/minar

print(get_vertex_dist(mesh))
get_vertex_dist(mesh_v4)


    

25.951030928079508


18.473870082480552